# Function calling - shell execution
## Introduction
Obviously we can also have OpenAI function execute shell commands.

## Installation

In [1]:
%pip install -q openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Shell functions

In [2]:
import subprocess
import json

# Function to execute shell commands and pass the output
# Note that if it's an error it will also pass the error.
def execute_shell_script(script_content):
    """
    Execute the given shell script content and return the output.
    """
    try:
        result = subprocess.run(['bash', '-c', script_content], 
                                capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"

## Helper function to return the output as json
def run_shell_script(script_content):
    """
    OpenAI function to run a shell script and return the output.
    """
    output = execute_shell_script(script_content)
    return json.dumps({"output": output})

We can run the functions directly.

In [3]:
# Execute a shell command directly
print("\nExecuting shell command directly:")
direct_output = execute_shell_script("echo 'Hello from the shell!' && date")
print(direct_output)
direct_output = execute_shell_script("docker -h")
print(direct_output)


Executing shell command directly:
Hello from the shell!
Thu Mar 13 11:06:08 CDT 2025


Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Common Commands:
  run         Create and run a new container from an image
  exec        Execute a command in a running container
  ps          List containers
  build       Build an image from a Dockerfile
  pull        Download an image from a registry
  push        Upload an image to a registry
  images      List images
  login       Authenticate to a registry
  logout      Log out from a registry
  search      Search Docker Hub for images
  version     Show the Docker version information
  info        Display system-wide information

Management Commands:
  ai*         Ask Gordon - Docker Agent
  builder     Manage builds
  buildx*     Docker Buildx
  checkpoint  Manage checkpoints
  compose*    Docker Compose
  container   Manage containers
  context     Manage contexts
  debug*      Get a shell into any image or contai

## Setting the functions for OpenAI

In [4]:
# Define the function for OpenAI
functions = [
    {
        "name": "run_shell_script",
        "description": "Execute a shell script and return the output",
        "parameters": {
            "type": "object",
            "properties": {
                "script_content": {
                    "type": "string",
                    "description": "The content of the shell script to execute"
                }
            },
            "required": ["script_content"]
        }
    }
]

# Example conversation
messages = [
    {"role": "system", "content": "You are a helpful assistant that can execute shell scripts."},
    {"role": "user", "content": "Can you run a shell script that lists the contents of the current directory?"}
]

In [5]:
from openai import OpenAI

# Setup the connection
client = OpenAI()

# Get the completion from OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto"
)

In [6]:
# Extract the function call from the response
message = response.choices[0].message

if message.function_call:
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    
    if function_name == 'run_shell_script':
        # Here we're explicitly setting the script to list directory contents
        print("Running shell script with the following content:")
        print(function_args)

        # To avoid any risks, we override it here , to make it safe
        script_content = "ls -la"
        result = run_shell_script(script_content)
        print("==============================")
        print(f"Script output: {result}")
else:
    print(message.content)

Running shell script with the following content:
{'script_content': 'ls -la'}
Script output: {"output": "total 256\ndrwxr-xr-x@ 14 johnwillis  staff    448 Mar 13 10:07 \u001b.\u001b[m\u001b[m\ndrwxr-xr-x   8 johnwillis  staff    256 Mar 13 10:07 \u001b..\u001b[m\u001b[m\n-rw-r--r--@  1 johnwillis  staff   3544 Mar 13 10:08 01-simple-json.ipynb\n-rw-r--r--@  1 johnwillis  staff  11413 Mar 13 11:00 02-pydantic-json.ipynb\n-rw-r--r--@  1 johnwillis  staff  13266 Mar 13 10:07 03-parsing-retry.ipynb\n-rw-r--r--@  1 johnwillis  staff   4645 Mar 13 10:07 04-openai-structured.ipynb\n-rw-r--r--@  1 johnwillis  staff   9870 Mar 13 10:49 05-openai-function_calling.ipynb\n-rw-r--r--@  1 johnwillis  staff   4464 Mar 13 10:07 06-langchain-function-calling.ipynb\n-rw-r--r--@  1 johnwillis  staff  20756 Mar 13 10:07 07-apichain-openapi.ipynb\n-rw-r--r--@  1 johnwillis  staff   4443 Mar 13 10:07 08-function-example1.ipynb\n-rw-r--r--@  1 johnwillis  staff  13108 Mar 13 10:07 09-function-example2.ipynb